# Parameters List
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(1.0, 4.5, 0.5))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [4]:
len(all_combinations)

28

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('hyperplane_nondrifted.csv')

In [11]:
data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.685360,0.953393,0.003948,0.512192,0.812621,0.612526,0.721755,0.291876,0.917774,0.714576,1
1,0.142170,0.373341,0.674134,0.441833,0.434014,0.617767,0.513138,0.650397,0.601039,0.805223,0
2,0.908649,0.319236,0.090459,0.300700,0.113984,0.828681,0.046896,0.626287,0.547586,0.819287,0
3,0.856850,0.351653,0.754648,0.295962,0.883936,0.325512,0.165016,0.392529,0.093460,0.821106,1
4,0.384114,0.944261,0.987625,0.456305,0.826123,0.251374,0.597372,0.902832,0.534558,0.590201,1
...,...,...,...,...,...,...,...,...,...,...,...
39995,0.061714,0.462585,0.943864,0.143777,0.657776,0.325195,0.040887,0.946246,0.313470,0.068780,0
39996,0.073441,0.542813,0.416149,0.585142,0.912097,0.400436,0.582935,0.003902,0.662642,0.179987,0
39997,0.686396,0.536559,0.136439,0.324377,0.606774,0.788541,0.472763,0.443855,0.809508,0.259377,0
39998,0.871673,0.893466,0.046246,0.231087,0.316337,0.445972,0.039865,0.051370,0.502055,0.350849,0


In [12]:
drifted_data=pd.read_csv('hyperplane_drifted.csv')

In [13]:
drifted_data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.855904,0.753047,0.804241,0.623340,0.608495,0.418715,0.162841,0.105965,0.589660,0.829179,1
1,0.784414,0.408043,0.259734,0.855947,0.919578,0.984292,0.186262,0.663626,0.482852,0.981291,1
2,0.658460,0.033933,0.713858,0.378366,0.906390,0.875112,0.190273,0.245564,0.062429,0.841749,1
3,0.136558,0.231648,0.482025,0.693201,0.245718,0.882909,0.635738,0.522973,0.960885,0.520092,0
4,0.152298,0.665623,0.595665,0.101269,0.544626,0.456066,0.155814,0.648949,0.516301,0.073947,0
...,...,...,...,...,...,...,...,...,...,...,...
2555,0.849341,0.442411,0.238075,0.930455,0.851710,0.134304,0.680624,0.941170,0.695621,0.203228,1
2556,0.333330,0.205006,0.313797,0.816728,0.412498,0.847833,0.961956,0.100099,0.973875,0.380310,0
2557,0.695616,0.735272,0.286571,0.622903,0.656901,0.127594,0.368878,0.212568,0.305806,0.933580,1
2558,0.534210,0.876190,0.020611,0.358940,0.932018,0.659357,0.118251,0.273765,0.597341,0.027397,1


In [14]:
del drifted_data['output']

In [15]:
drifted_data=drifted_data[0:1280]

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [16]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative  

In [17]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [18]:
def autoencoder (train,epochs,val_set):
    
    # set bottleneck 1/3 of the input layer size
    #encoding_dim = int(np.floor(train.shape[-1]/3))
       
    # input layer size= # of attributes in the dataset after one-hot encoding
    
    
    optimizer = tensorflow.keras.optimizers.Adam()
    kernel_regularizer=None    
    learning_rate = 0.001  # Extract the learning rate
    optimizer.learning_rate = learning_rate
   
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded=layers.Dense(4,activation='relu')(input_layer) # Bottleneck
    decoded = layers.Dense(train.shape[1], activation='sigmoid',kernel_regularizer=kernel_regularizer)(encoded) # Output Layer

    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    
    
   
    # above model maps an input to its reconstruction 
   
    autoencoder.compile(optimizer=optimizer, loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [19]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [20]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
394/394 [==============================] - 2s 5ms/step - loss: 0.0807 - val_loss: 0.0710
Epoch 2/100
394/394 [==============================] - 1s 1ms/step - loss: 0.0696 - val_loss: 0.0671
Epoch 3/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0662 - val_loss: 0.0651
Epoch 4/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0642 - val_loss: 0.0639
Epoch 5/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0630 - val_loss: 0.0633
Epoch 6/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0623 - val_loss: 0.0630
Epoch 7/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0619 - val_loss: 0.0628
Epoch 8/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0620 - val_loss: 0.0626
Epoch 9/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0619 - val_loss: 0.0626
Epoch 10/100
394/394 [==========================

Epoch 81/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0520 - val_loss: 0.0528
Epoch 82/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0519 - val_loss: 0.0528
Epoch 83/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0519 - val_loss: 0.0528
Epoch 84/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0515 - val_loss: 0.0527
Epoch 85/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0518 - val_loss: 0.0528
Epoch 86/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0516 - val_loss: 0.0527
Epoch 87/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0519 - val_loss: 0.0528
Epoch 88/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0514 - val_loss: 0.0528
Epoch 89/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0522 - val_loss: 0.0528
Epoch 90/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0520

Epoch 61/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0436 - val_loss: 0.0440
Epoch 62/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437 - val_loss: 0.0441
Epoch 63/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0440 - val_loss: 0.0439
Epoch 64/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0440 - val_loss: 0.0440
Epoch 65/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0439 - val_loss: 0.0440
Epoch 66/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0440 - val_loss: 0.0440
Epoch 67/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0436 - val_loss: 0.0439
Epoch 68/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0438 - val_loss: 0.0439
Epoch 69/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0441 - val_loss: 0.0440
Epoch 70/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437

In [21]:
test

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
28000,0.571178,0.497160,0.018756,0.502050,0.557763,0.803582,0.907354,0.147472,0.518609,0.095038,0
28001,0.939351,0.968282,0.897398,0.100867,0.604813,0.217409,0.583915,0.184439,0.359195,0.747322,1
28002,0.568025,0.063038,0.448904,0.528824,0.031582,0.365676,0.230242,0.637741,0.797741,0.579740,0
28003,0.674628,0.832663,0.856485,0.918181,0.055832,0.046599,0.980739,0.832307,0.410864,0.316403,1
28004,0.175722,0.734974,0.752364,0.905388,0.731960,0.474822,0.546828,0.394240,0.824517,0.384729,1
...,...,...,...,...,...,...,...,...,...,...,...
35995,0.148484,0.866953,0.314443,0.207260,0.094772,0.872350,0.213283,0.271287,0.383104,0.363098,0
35996,0.445462,0.547033,0.616315,0.296604,0.238751,0.625008,0.547060,0.625499,0.177021,0.910157,0
35997,0.237011,0.265461,0.253018,0.387041,0.126623,0.996028,0.675586,0.104079,0.722377,0.356402,0
35998,0.222569,0.961094,0.395693,0.889444,0.226125,0.332889,0.762653,0.037542,0.900588,0.050933,0


## Validation Data

In [22]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [23]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['output']==1]
    test_neg_class=test[test['output']==0]
    del test_pos_class['output']
    del test_neg_class['output']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [24]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 

In [25]:
k_parameter=list(np.arange(1.0, 4.5, 0.5))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [26]:
len(all_combinations)

28

### AEDDM Framwework

In [27]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [28]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [29]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [30]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [31]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=100):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['output']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
drifted=drifted_data[0:1280]
batches_d=make_batches(drifted)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    

    
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.5))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [32]:
results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.05484936281679751 , Count Threshold: 9.0
Thresholds for Nagative AE
Batch Thresold: 0.04576969223864502 , Count Threshold: 9.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 7, 12, 20, 21, 22, 23, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 6, 7, 9, 10, 12, 15, 16, 18, 19, 20, 21, 25, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 7, 10, 12, 13, 16, 21, 24, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 4, 6, 7, 10, 12, 14, 16, 17, 19, 20, 21, 23, 25, 26, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 2

Data Points Exceeding L


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 4, 9, 10, 12, 13, 16, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 4, 5, 7, 9, 11, 12, 18, 19, 22, 23, 24, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 4, 7, 13, 14, 16, 24, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 13



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 5, 11, 12, 16, 17, 18, 22, 23, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 8, 9, 11, 14, 15, 18, 24, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 4


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 7, 12, 20, 21, 22, 23, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 6, 7, 9, 10, 12, 15, 16, 18, 19, 20, 21, 25, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 7, 10, 12, 13, 16, 21, 24, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 4, 6, 7, 10, 12, 14, 16, 17, 19, 20, 21, 23, 25, 26, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 7, 8, 9, 14, 17, 18, 21, 22, 23, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 5, 9, 11, 12, 13, 14, 17, 20, 22, 23, 24, 25, 26, 27, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 4, 5, 7, 9, 11, 12, 18, 19, 22, 23, 24, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 4, 7, 13, 14, 16, 24, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 13



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 5, 11, 12, 16, 17, 18, 22, 23, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 8, 9, 11, 14, 15, 18, 24, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 4, 5, 7, 12, 13, 19, 22, 25, 26, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 7, 9, 10, 11, 12, 16, 20, 22, 23, 25, 27, 28, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 28




Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 7, 20, 22, 23, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 7, 10, 12, 13, 21, 24, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 7, 8, 9, 14, 18, 21, 23, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 8, 17, 19, 21, 25, 27, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [10, 16] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Thresh


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 16] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [8] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Thres


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 



Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : 

In [33]:
results_df_p

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.5,median,and,17,17,0,7,7,90
1,0.5,median,or,17,17,0,7,7,90
2,0.5,max,and,0,0,20,0,0,60
3,0.5,max,or,2,2,20,0,0,54
4,1.0,median,and,14,14,0,7,7,99
5,1.0,median,or,14,14,0,10,10,108
6,1.0,max,and,0,0,10,1,1,93
7,1.0,max,or,3,3,10,2,2,87
8,1.5,median,and,12,12,0,4,4,96
9,1.5,median,or,14,14,0,5,5,93


In [34]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [35]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Categorical, Real


def aeddm_function(x):
    global results_df
    row=0
    k,alpha,logical=x
    values=[k, alpha, logical]
    #results_df.iloc[row :3] = values
        
    print("***************************************************")
    print(k,alpha,logical)
    parameters=(k,alpha,logical)
    instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
    instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
    #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
    # Counts the MSE values exceeding threshold in each batch 
    exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
    exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
    count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
    count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
    print("Thresholds for Positive AE")
    #print(alpha)
    print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
    print("Thresholds for Nagative AE")
    print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
    # AEDDM Starts Here 
                
    thres_zscore_batch_pos=batch_thresh_pos
    thres_zscore_batch_neg=batch_thresh_neg
                             
                
    #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
        #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
             #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    print(" Drifted Data Processing Starts Here")
                
    score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    #value+=100
    print(col_values)
    print(values)
    values.extend(col_values)
    print(values)
    
    results_df = results_df.append(pd.Series(values, index=results_df.columns), ignore_index=True)
    print(results_df)
    
    values=[]
    col_values=[]
    print(k,alpha,logical)
    return score 



In [36]:
# Define the search space
row=0
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    #aeddm_function,
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=50,  # Number of iterations
    acq_func='EI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.06797383331229845 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.05879144848250966 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshol


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 10, 13, 14, 23, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 12, 15, 17, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 15, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 10, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 8, 20, 22, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 24, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 11, 13, 19, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 7, 14, 21]

Number of Data Points Exceeding Layer 2 Encode


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [8

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 8, 13, 16, 24, 26, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 6, 8, 15, 16, 17, 19, 20, 22, 23, 27, 28, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 10, 14, 16, 20, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 16, 21, 23, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 7, 20, 23, 24, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 19, 21, 22, 24, 31]


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 4, 5, 6, 15, 17, 18, 22, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 11, 17, 18, 21, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 7



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 7, 8, 10, 11, 14, 16, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [24, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 8, 10, 17, 18, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [7, 11, 12, 15, 16, 22, 27]

Number 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 8, 12, 14, 15, 17, 19, 21, 24, 25, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 6, 8, 15, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 9, 11, 13, 17, 23, 27, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 13, 16, 23, 27, 28, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 12, 14, 19, 20, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 9, 16, 18, 19]

Number


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 7, 11, 17, 20, 22, 23, 24, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 5, 8, 9, 10, 26, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 7, 11, 17, 18, 19, 22, 23, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 4, 22, 23, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 5, 13, 21, 24, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 4, 6, 10, 12, 13, 16, 18, 20, 21, 23, 24, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 7, 8, 12, 14, 18, 25, 29


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 7, 11, 19] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 12, 14, 19, 20, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 5, 9, 16, 18, 19, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8

 Drift Detection at Batch Level

 Threshold exceeds at batch : 0
[0]
Warning Level at Batch 0
 Threshold exceeds at batch : 1
[0, 1]
Warning Level at Batch 1
 Threshold exceeds at batch : 2
[0, 1, 2]
 Drift Confirmed at Batch No :  0
 Threshold exceeds at batch : 3
[0, 1, 2, 3]
 Drift Confirmed at Batch No :  1
 Threshold exceeds at batch : 4
[0, 1, 2, 3, 4]
 Drift Confirmed at Batch No :  2
 Threshold exceeds at batch : 5
[0, 1, 2, 3, 4, 5]
 Drift Confirmed at Batch No :  3
 Threshold exceeds

In [37]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -108
Best parameters: [0.9785749237063, 'median', 'or']


In [38]:
df_EI=results_df.copy()

In [39]:
df_EI

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,20,0,0,60
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,12,12,1,4,4,93
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,20,0,0,60
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,0,0,20,0,0,60
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,0,0,20,0,0,60
9,1.455675,max,or,0,0,20,0,0,60


In [40]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [41]:
# Define the search space
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='PI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.06797383331229845 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.05879144848250966 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshol


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 10, 13, 14, 23, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 12, 15, 17, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 15, 21, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 10, 15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 8, 20, 22, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 24, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 11, 13, 19, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 7, 14, 21]

Number of Data Points Exceeding Layer 2 Encode


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [8

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 8, 13, 16, 24, 26, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 6, 8, 15, 16, 17, 19, 20, 22, 23, 27, 28, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 10, 14, 16, 20, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 21, 23, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 7, 20, 23, 24, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 19, 22, 24, 31]

Number


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 7, 12, 14, 15] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 12, 13, 20, 23, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 6

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 4, 5, 6, 12, 13, 15, 16, 17, 18, 21, 22, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 7, 10, 11, 17, 18, 21, 23, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 11



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 7, 8, 10, 11, 14, 16, 19, 31] 

Data Points Exceeding Layer 2 Encoder Insta


Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 10, 14, 16, 20, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 21, 23, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 7, 20, 23, 24, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 19, 22, 24, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 8, 12, 14, 15, 17, 19, 21, 24, 25, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 5, 6, 7, 8, 10, 15, 22, 23, 29]

Number of Dat


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 14, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 4, 5, 6, 12, 13, 15, 16, 17, 18, 21, 22, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 7, 10, 11, 17, 18, 21, 23, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 11



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 7, 8, 10, 11, 14, 16, 19, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [24, 25, 26, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 8, 10, 17, 18, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder I


Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 21, 23, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 7, 20, 23, 24, 26, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 19, 22, 24, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 8, 12, 14, 15, 17, 19, 21, 24, 25, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 5, 6, 7, 8, 10, 15, 22, 23, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 7, 8, 9, 11, 13, 16, 17, 21, 23, 27, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Thr

In [42]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -108
Best parameters: [1.3274294459703357, 'median', 'and']


In [43]:
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,20,0,0,60
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,12,12,1,4,4,93
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,20,0,0,60
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,0,0,20,0,0,60
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,0,0,20,0,0,60
9,1.455675,max,or,0,0,20,0,0,60


### Testing Code

In [44]:
Thresh=0.212964
count_thresh=0
logical='and'

In [45]:
drift_point=20
WnDs= 0 
WdDs=0 
DnDs=0          
delay=None     
DdDs=0   

In [50]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    value=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={value}")
    
    

In [51]:
detect_drift2(avg_mse_l2_list_d,exceed_count_L2_countThresh_d, Thresh,count_thresh,logical)

Number of Drifted Batches0
All Drifted Batches:  []
Drifted Batches before Drift Point:  []
Drifted Batches after Drift Point:  []
WnDs=0 ,WdDs=0 , DnDs=0,DdDs=0 , delay= 20 , value=60
